In [ ]:
%matplotlib inline
import numpy as np
import glob
import matplotlib.pyplot as plt

In [ ]:
def get_SNR(file):
    with open(file, "rt")as f:
        a = f.readlines()
    lst = []
    for x in a:
        if "Value:" in x:
            lst.append(float(x.split("Value: ")[1]))
        #print(x.split("bla"))
    lst = lst[len(lst)//2 : ]
    return (np.array(lst).mean(), np.array(lst).std())

In [ ]:
def get_BER(file):
    with open(file, "rb")as f:
        _ = f.read()
    
    rcv = np.frombuffer(_, dtype = np.uint8)
    aux = np.unpackbits(rcv)
    
    tx = np.array([0, 255, 40, 250, 37, 42, 186], dtype = np.uint8)
    
    pos = np.argmax(np.correlate(np.unpackbits(rcv), np.unpackbits(tx)))
    rcv = np.packbits(np.roll(np.unpackbits(rcv), -pos))
    tx_extended = np.tile(tx, int(rcv.shape[0]/len(tx)))
    
    return (1 - np.mean(np.equal(np.unpackbits(rcv), np.unpackbits(tx_extended))))

In [ ]:
def noDiffEnc_Corrector(file):
    with open(file, "rb")as f:
        _ = f.read()
    
    rcv = np.frombuffer(_, dtype = np.uint8)
    
    tx = np.array([0, 255, 40, 250, 37, 42, 186], dtype = np.uint8)
    
    pos = np.argmax(np.correlate(np.unpackbits(rcv), np.unpackbits(tx)))
    rcv = np.packbits(np.roll(np.unpackbits(rcv), -pos))
    
    new_arr = []
    tx_inverted = np.invert(tx)
    rcv_splitted = np.split(rcv, len(rcv)/len(tx))
    for x in rcv_splitted:
        comp1 = np.mean(np.equal(np.unpackbits(tx), np.unpackbits(x)))
        comp2 = np.mean(np.equal(np.unpackbits(tx_inverted), np.unpackbits(x)))
        if comp2 > comp1:
            new_arr.append(np.invert(x))
        else:
            new_arr.append(x)
    final_array = np.array(new_arr, dtype = np.uint8).ravel()
    
    tx_extended = np.tile(tx, int(rcv.shape[0]/len(tx)))
    return (1 - np.mean(np.equal(np.unpackbits(final_array), np.unpackbits(tx_extended))))

In [ ]:
%%time
get_SNR("output.txt"), noDiffEnc_Corrector("output.data")

## <font color = red> Quanto mais baixo o SNR, maior a chance de ocorrer problemas na recepção, mais chance do resultado obtido ser ruim
    
# <font color = blue> Resultados obtidos simulados
  <font color = blue> SNR Média, SNR STD, BER


In [ ]:
a = """
"""

In [ ]:
snr_avgs = []
snr_stds = []
bers = []

for line in a.splitlines():
    aux = line.replace("(", "").replace(")", "").split(",")
    if aux != [""]:
        snr_avgs.append(float(aux[0]))
        snr_stds.append(float(aux[1]))
        bers.append(float(aux[2]))

In [ ]:
def print_lst_with_slash(lst):
    print("[ \\")
    for x in lst:
        print(f"{x}, \\")
    print("]")

In [ ]:
EbN0_min = 0
EbN0_max = 10
EbN0_range = range(EbN0_min, EbN0_max+1)

In [ ]:
simu = np.load("simu_bpsk.npy")

In [ ]:
theory = np.load("theory_bpsk.npy")

In [ ]:
print(plt.style.available)

In [ ]:
plt.style.use('ggplot')
plt.semilogy(EbN0_range, theory, "g-.", label = "BPSK Teórico")
plt.semilogy(EbN0_range, simu, "b-o", label = "BPSK Simulado")
plt.errorbar(snr_avgs, bers, fmt = "o", color = "yellow", label = "Simu/Prac BPSK")
plt.legend()
plt.grid()

plt.title('Simulação BER - BPSK')
plt.xlabel('Eb/N0 (dB)')
plt.ylabel('BER')
plt.savefig("Curva.png", dpi = 100)